In [ ]:
from pathlib import Path
import shutil


path_root = Path("data/cracks_pavement")
path_broken = path_root / "broken"
path_broken.mkdir(exist_ok=True)
paths_folders = sorted(p for p in path_root.iterdir() if p.name.startswith("SESSION"))

def rename_file(p: Path) -> None:
    stem = p.stem.split("-")[0] if "-" in p.stem else p.stem
    if stem.startswith("out") and stem[3:].isdigit():
        num = int(stem[3:])
        new_stem = f"{num:05d}"
        new_path = p.with_stem(new_stem)
        shutil.move(p, new_path)

for path_folder in paths_folders:
    for subfolder in ("img", "mask", "target"):
        for p in (path_folder / subfolder).iterdir():
            rename_file(p)

    path_img = path_folder / "img"
    path_mask = path_folder / "mask"
    path_target = path_folder / "target"

    names_img = {p.stem for p in path_img.iterdir()}
    names_mask = {p.stem for p in path_mask.iterdir()}
    names_target = {p.stem for p in path_target.iterdir()}

    all_names = names_img | names_mask | names_target

    for name in all_names:
        present = {
            "img": name in names_img,
            "mask": name in names_mask,
            "target": name in names_target,
        }
        if not all(present.values()):
            for subfolder, is_present in present.items():
                if is_present:
                    src = path_folder / subfolder / f"{name}.png"
                    dst = path_broken / path_folder.name / subfolder
                    dst.mkdir(parents=True, exist_ok=True)
                    shutil.move(src, dst / src.name)
